Logistic regression is a generalized linear model that we can use to model or predict categorical outcome variables. If the value is above the threshold limit, the model assigns the regression value as 1, otherwise it is assigned 0.

In logistic regression, we're essentially trying to find the weights that maximize the likelihood of producing our given data.

In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# simulated data
np.random.seed(12)
num_observations = 5000

# simulate separable data by sampling from a multivariate normal distribution
x1 = np.random.multivariate_normal([0, 0], [[1, .75],[.75, 1]], num_observations)
x2 = np.random.multivariate_normal([1, 4], [[1, .75],[.75, 1]], num_observations)

simulated_separableish_features = np.vstack((x1, x2)).astype(np.float32)
simulated_labels = np.hstack((np.zeros(num_observations),
                              np.ones(num_observations)))

In [ ]:
# plotting the data
plt.figure(figsize=(12,8))
plt.scatter(simulated_separableish_features[:, 0], simulated_separableish_features[:, 1],
            c = simulated_labels, alpha = .4)

In [ ]:
# choosing link function
# we use sigmoid here to transform linear model of predictors

def sigmoid(scores):
    return 1 / (1 + np.exp(-scores))

The likelihood (for binary classification) can be reduced to a fairly intuitive form by switching to the log-likelihood. This can be done without affecting the weights parameter estimation because log transformation are monotonic.

#### Calculating the Log-Likelihood

The log-likelihood can be viewed as as sum over all the training data:

$$\begin{equation}
ll = \sum_{i=1}^{N}y_{i}\beta ^{T}x_{i} - log(1+e^{\beta^{T}x_{i}})
\end{equation}$$

where $y$ is the target class, $x_{i}$ represents an individual data point, and $\beta$ is the weights vector.


In [ ]:
# maximize likelihood by computing likelihood and gradient


def log_likelihood(features, target, weights):
    scores = np.dot(features, weights)
    ll = np.sum( target*scores - np.log(1 + np.exp(scores)) )
    return ll

In [ ]:
# building the regression function

def logistic_regression(features, target, num_steps, learning_rate, add_intercept = False):
    if add_intercept:
        intercept = np.ones((features.shape[0], 1))
        features = np.hstack((intercept, features))
        
    weights = np.zeros(features.shape[1])
    
    for step in xrange(num_steps):
        scores = np.dot(features, weights)
        predictions = sigmoid(scores)

        # Update weights with log likelihood gradient
        output_error_signal = target - predictions
        
        gradient = np.dot(features.T, output_error_signal)
        weights += learning_rate * gradient

        # Print log-likelihood every so often
        if step % 10000 == 0:
            print log_likelihood(features, target, weights)
        
    return weights

Time to do the regression.

In [ ]:
# perform regression

weights = logistic_regression(simulated_separableish_features, simulated_labels,
                     num_steps = 50000, learning_rate = 5e-5, add_intercept=True)

In [ ]:
print weights

In [ ]:
# Comparing with sklearn logistic regression model

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(fit_intercept=True, C = 1e15)  # such value of c reduces regularization to zero
clf.fit(simulated_separableish_features, simulated_labels)

In [ ]:
print clf.intercept_, clf.coef_
print weights

Gradient descent on a convex function will always reach the global optimum, given enough time and sufficiently small learning rate.

In [ ]:
# calculating accuracy

final_scores = np.dot(np.hstack((np.ones((simulated_separableish_features.shape[0], 1)),
                                 simulated_separableish_features)), weights)
preds = np.round(sigmoid(final_scores))

print 'Accuracy from scratch: {0}'.format((preds == simulated_labels).sum().astype(float) / len(preds))
print 'Accuracy from sk-learn: {0}'.format(clf.score(simulated_separableish_features, simulated_labels))

In [ ]:
# plotting the outcomes

plt.figure(figsize = (12, 8))
plt.scatter(simulated_separableish_features[:, 0], simulated_separableish_features[:, 1],
            c = preds == simulated_labels - 1, alpha = .8, s = 50)